# Collection of data
As a starting point, a set of over 100,000 named chemical structures was downloaded from ZINC (https://zinc.docking.org/).
Using pubchem's pugrest apis, the structures were matched with CID and predicted logp values (xlogp3).
The CID can be mapped to DSSTox for filling in experimental values.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('named.csv')

In [3]:
df.head()

,Unnamed: 0,zinc_id,smiles,cid,Canonicalized Compound,Hydrogen Bond Acceptor Count,Hydrogen Bond Donor Count,Rotatable Bond Count,Allowed IUPAC Name,CAS-like Style IUPAC Name,...,Standard InChI,Standard InChIKey,XLogP3-AA Log P,Exact Mass,Canonical SMILES,Isomeric SMILES,Polar Surface Area Topological,MonoIsotopic Weight,XLogP3 Log P,logp_experimental
0,0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,9850878.0,1.0,7.0,3.0,10.0,"[(1S,2R,3S,4S,6R,7R,8R,14R)-3-hydroxy-2,4,7,14...",2-[[1-[[(2R)-2-amino-3-methyl-1-oxobutyl]amino...,...,InChI=1S/C31H52N2O5S/c1-10-29(8)15-22(38-23(35...,LLYYNOVSVPBRGV-MVNKZKPCSA-N,5.3,564.359694,CC1CCC23CCC(=O)C2C1(C(CC(C(C3C)O)(C)C=C)OC(=O)...,C[C@@H]1CC[C@@]23CCC(=O)[C@H]2[C@@]1([C@@H](C[...,144.0,564.359694,NaN,NaN
1,1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,9971439.0,1.0,3.0,1.0,8.0,(2S)-2-[bis(2-chloroethyl)amino]-3-phenyl-prop...,(2S)-2-[bis(2-chloroethyl)amino]-3-phenylpropa...,...,InChI=1S/C13H17Cl2NO2/c14-6-8-16(9-7-15)12(13(...,QEGIMPLUXVXFNQ-LBPRGKRZSA-N,0.8,289.063634,C1=CC=C(C=C1)CC(C(=O)O)N(CCCl)CCCl,C1=CC=C(C=C1)C[C@@H](C(=O)O)N(CCCl)CCCl,40.5,289.063634,NaN,NaN
4,4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import urllib.request
import urllib.parse
import json
import re
import statistics

In [5]:
def extract_value(info):
    if ('Number' in info['Value']):
        return float(info['Value']['Number'][0])
    elif ('StringWithMarkup' in info['Value']):
        match = re.search('(-?\d+\.?\d{0,4})', info['Value']['StringWithMarkup'][0]['String'])
        if (match):
            return float(match.group())
    return None

In [9]:
def get_experimental_logp(cid):
    try:
        response = urllib.request.urlopen(
            "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/JSON/?heading=LogP" % str(cid)).read()
        data = json.loads(response.decode('utf-8'))
        values = map(extract_value, data['Record']['Section'][0]['Section'][0]['Section'][0]['Information'])
        return round(statistics.mean(filter(lambda v: v is not None, values)), 2)
    except Exception as e:
        print('%s (%s)' % (str(cid), e))
        pass
    return ''

In [11]:
get_experimental_logp(6993105)

4.64

In [13]:
import time
import numpy as np

def throttled_pubchem_logp(cid):
    if (not cid or np.isnan(cid)):
        return None
    start = time.time()
    result = get_experimental_logp(str(int(cid)))
    end = time.time()
    elapsed = end - start
    if (elapsed < 0.2):
        time.sleep(0.2 - elapsed)
    return result

In [14]:
df['logp_experimental'] = df['cid'].apply(throttled_pubchem_logp)
df_combined.isna().sum()

9850878 (HTTP Error 404: PUGVIEW.NotFound)
9971439 (HTTP Error 404: PUGVIEW.NotFound)
26250142 (HTTP Error 404: PUGVIEW.NotFound)
12313370 (HTTP Error 404: PUGVIEW.NotFound)
13696363 (HTTP Error 404: PUGVIEW.NotFound)
5460495 (HTTP Error 404: PUGVIEW.NotFound)
76965413 (HTTP Error 404: PUGVIEW.NotFound)
6428990 (HTTP Error 404: PUGVIEW.NotFound)
76312362 (HTTP Error 404: PUGVIEW.NotFound)
101164130 (HTTP Error 404: PUGVIEW.NotFound)
656778 (HTTP Error 404: PUGVIEW.NotFound)
14287396 (HTTP Error 404: PUGVIEW.NotFound)
76960772 (HTTP Error 404: PUGVIEW.NotFound)
124524289 (HTTP Error 404: PUGVIEW.NotFound)
160400 (HTTP Error 404: PUGVIEW.NotFound)
122239 (HTTP Error 404: PUGVIEW.NotFound)
44139369 (HTTP Error 404: PUGVIEW.NotFound)
207845 (HTTP Error 404: PUGVIEW.NotFound)
36688525 (HTTP Error 404: PUGVIEW.NotFound)
22295157 (HTTP Error 404: PUGVIEW.NotFound)
44219741 (HTTP Error 404: PUGVIEW.NotFound)
154497855 (HTTP Error 404: PUGVIEW.NotFound)
53355889 (HTTP Error 404: PUGVIEW.NotFoun

124356128 (HTTP Error 404: PUGVIEW.NotFound)
148815 (HTTP Error 404: PUGVIEW.NotFound)
86311083 (HTTP Error 404: PUGVIEW.NotFound)
5316762 (HTTP Error 404: PUGVIEW.NotFound)
10720367 (HTTP Error 404: PUGVIEW.NotFound)
101969151 (HTTP Error 404: PUGVIEW.NotFound)
177936 (HTTP Error 404: PUGVIEW.NotFound)
7064767 (HTTP Error 404: PUGVIEW.NotFound)
4619 (HTTP Error 404: PUGVIEW.NotFound)
29979359 (HTTP Error 404: PUGVIEW.NotFound)
23247869 (HTTP Error 404: PUGVIEW.NotFound)
2608 (HTTP Error 404: PUGVIEW.NotFound)
44559683 (HTTP Error 404: PUGVIEW.NotFound)
446275 (HTTP Error 404: PUGVIEW.NotFound)
11390334 (HTTP Error 404: PUGVIEW.NotFound)
76322784 (HTTP Error 404: PUGVIEW.NotFound)
87989055 (HTTP Error 404: PUGVIEW.NotFound)
9979609 (HTTP Error 404: PUGVIEW.NotFound)
23642503 (HTTP Error 404: PUGVIEW.NotFound)
9978441 (HTTP Error 404: PUGVIEW.NotFound)
26500368 (HTTP Error 404: PUGVIEW.NotFound)
167 (HTTP Error 404: PUGVIEW.NotFound)
25147318 (HTTP Error 404: PUGVIEW.NotFound)
46881102 

10587535 (HTTP Error 404: PUGVIEW.NotFound)
86308795 (HTTP Error 404: PUGVIEW.NotFound)
10098460 (HTTP Error 404: PUGVIEW.NotFound)
1628 (HTTP Error 404: PUGVIEW.NotFound)
76959803 (HTTP Error 404: PUGVIEW.NotFound)
445155 (HTTP Error 404: PUGVIEW.NotFound)
15788490 (HTTP Error 404: PUGVIEW.NotFound)
25011726 (HTTP Error 404: PUGVIEW.NotFound)
131752175 (HTTP Error 404: PUGVIEW.NotFound)
25243459 (HTTP Error 404: PUGVIEW.NotFound)
4369374 (HTTP Error 404: PUGVIEW.NotFound)
71524786 (HTTP Error 404: PUGVIEW.NotFound)
101944839 (HTTP Error 404: PUGVIEW.NotFound)
796 (HTTP Error 404: PUGVIEW.NotFound)
44605338 (HTTP Error 404: PUGVIEW.NotFound)
76961436 (HTTP Error 404: PUGVIEW.NotFound)
821442 (HTTP Error 404: PUGVIEW.NotFound)
45270450 (HTTP Error 404: PUGVIEW.NotFound)
86065201 (HTTP Error 404: PUGVIEW.NotFound)
11746161 (HTTP Error 404: PUGVIEW.NotFound)
16092094 (HTTP Error 404: PUGVIEW.NotFound)
92245213 (HTTP Error 404: PUGVIEW.NotFound)
11516262 (HTTP Error 404: PUGVIEW.NotFound)


76970225 (HTTP Error 404: PUGVIEW.NotFound)
3000323 (HTTP Error 404: PUGVIEW.NotFound)
10263201 (HTTP Error 404: PUGVIEW.NotFound)
10387872 (HTTP Error 404: PUGVIEW.NotFound)
42640644 (HTTP Error 404: PUGVIEW.NotFound)
24800710 (HTTP Error 404: PUGVIEW.NotFound)
51042625 (HTTP Error 404: PUGVIEW.NotFound)
44575187 (HTTP Error 404: PUGVIEW.NotFound)
44584028 (HTTP Error 404: PUGVIEW.NotFound)
6931745 (HTTP Error 404: PUGVIEW.NotFound)
6993490 (HTTP Error 404: PUGVIEW.NotFound)
44264634 (HTTP Error 404: PUGVIEW.NotFound)
10090476 (HTTP Error 404: PUGVIEW.NotFound)
6991987 (HTTP Error 404: PUGVIEW.NotFound)
801221 (HTTP Error 404: PUGVIEW.NotFound)
14558470 (HTTP Error 404: PUGVIEW.NotFound)
9808844 (HTTP Error 404: PUGVIEW.NotFound)
45269666 (HTTP Error 404: PUGVIEW.NotFound)
14217344 (HTTP Error 404: PUGVIEW.NotFound)
76971674 (HTTP Error 404: PUGVIEW.NotFound)
71314787 (HTTP Error 404: PUGVIEW.NotFound)
21674184 (HTTP Error 404: PUGVIEW.NotFound)
443408 (HTTP Error 404: PUGVIEW.NotFoun

54679224 (HTTP Error 404: PUGVIEW.NotFound)
25021441 (HTTP Error 404: PUGVIEW.NotFound)
127555 (HTTP Error 404: PUGVIEW.NotFound)
92271645 (HTTP Error 404: PUGVIEW.NotFound)
148724 (HTTP Error 404: PUGVIEW.NotFound)
11042673 (HTTP Error 404: PUGVIEW.NotFound)
445127 (HTTP Error 404: PUGVIEW.NotFound)
477163 (HTTP Error 404: PUGVIEW.NotFound)
60090780 (HTTP Error 404: PUGVIEW.NotFound)
66601447 (HTTP Error 404: PUGVIEW.NotFound)
3012473 (HTTP Error 404: PUGVIEW.NotFound)
30844631 (HTTP Error 404: PUGVIEW.NotFound)
6482976 (HTTP Error 404: PUGVIEW.NotFound)
44572330 (HTTP Error 404: PUGVIEW.NotFound)
53325890 (HTTP Error 404: PUGVIEW.NotFound)
44281346 (HTTP Error 404: PUGVIEW.NotFound)
42639666 (HTTP Error 404: PUGVIEW.NotFound)
15346771 (HTTP Error 404: PUGVIEW.NotFound)
6603957 (HTTP Error 404: PUGVIEW.NotFound)
67520884 (HTTP Error 404: PUGVIEW.NotFound)
92144248 (HTTP Error 404: PUGVIEW.NotFound)
16099434 (HTTP Error 404: PUGVIEW.NotFound)
93217017 (HTTP Error 404: PUGVIEW.NotFound)

...

98085553 (HTTP Error 404: PUGVIEW.NotFound)
9988319 (HTTP Error 404: PUGVIEW.NotFound)
9837867 (HTTP Error 404: PUGVIEW.NotFound)
4137083 (HTTP Error 404: PUGVIEW.NotFound)
71375650 (HTTP Error 404: PUGVIEW.NotFound)
17754112 (HTTP Error 404: PUGVIEW.NotFound)
24397 (HTTP Error 404: PUGVIEW.NotFound)
11000442 (HTTP Error 404: PUGVIEW.NotFound)
26183519 (HTTP Error 404: PUGVIEW.NotFound)
136229437 (HTTP Error 404: PUGVIEW.NotFound)
71577012 (HTTP Error 404: PUGVIEW.NotFound)
104838 (HTTP Error 404: PUGVIEW.NotFound)
46918647 (HTTP Error 404: PUGVIEW.NotFound)
76967813 (HTTP Error 404: PUGVIEW.NotFound)
44567192 (HTTP Error 404: PUGVIEW.NotFound)
10409797 (HTTP Error 404: PUGVIEW.NotFound)
44253993 (HTTP Error 404: PUGVIEW.NotFound)
136880 (HTTP Error 404: PUGVIEW.NotFound)
40466926 (HTTP Error 404: PUGVIEW.NotFound)
906172 (HTTP Error 404: PUGVIEW.NotFound)
6918293 (HTTP Error 404: PUGVIEW.NotFound)
21117006 (HTTP Error 404: PUGVIEW.NotFound)
13891923 (HTTP Error 404: PUGVIEW.NotFound)


101394434 (HTTP Error 404: PUGVIEW.NotFound)
68765 (HTTP Error 404: PUGVIEW.NotFound)
21123571 (HTTP Error 404: PUGVIEW.NotFound)
76968792 (HTTP Error 404: PUGVIEW.NotFound)
66593768 (HTTP Error 404: PUGVIEW.NotFound)
104909236 (HTTP Error 404: PUGVIEW.NotFound)
454092 (HTTP Error 404: PUGVIEW.NotFound)
303107 (HTTP Error 404: PUGVIEW.NotFound)
15939530 (HTTP Error 404: PUGVIEW.NotFound)
638391 (HTTP Error 404: PUGVIEW.NotFound)
38348556 (HTTP Error 404: PUGVIEW.NotFound)
158341 (HTTP Error 404: PUGVIEW.NotFound)
65624 (HTTP Error 404: PUGVIEW.NotFound)
157717 (HTTP Error 404: PUGVIEW.NotFound)
5319380 (HTTP Error 404: PUGVIEW.NotFound)
11060310 (HTTP Error 404: PUGVIEW.NotFound)
38988228 (HTTP Error 404: PUGVIEW.NotFound)
92201 (HTTP Error 404: PUGVIEW.NotFound)
6992319 (HTTP Error 404: PUGVIEW.NotFound)
23844143 (HTTP Error 404: PUGVIEW.NotFound)
53481014 (HTTP Error 404: PUGVIEW.NotFound)
59954486 (HTTP Error 404: PUGVIEW.NotFound)
8708 (HTTP Error 404: PUGVIEW.NotFound)
71866 (HTTP

92331687 (HTTP Error 404: PUGVIEW.NotFound)
21663858 (HTTP Error 404: PUGVIEW.NotFound)
46209786 (HTTP Error 404: PUGVIEW.NotFound)
98138852 (HTTP Error 404: PUGVIEW.NotFound)
146222 (HTTP Error 404: PUGVIEW.NotFound)
114810 (HTTP Error 404: PUGVIEW.NotFound)
44581486 (HTTP Error 404: PUGVIEW.NotFound)
10745604 (HTTP Error 404: PUGVIEW.NotFound)
3034881 (HTTP Error 404: PUGVIEW.NotFound)
55299939 (HTTP Error 404: PUGVIEW.NotFound)
12073719 (HTTP Error 404: PUGVIEW.NotFound)
446586 (HTTP Error 404: PUGVIEW.NotFound)
26650 (HTTP Error 404: PUGVIEW.NotFound)
71224 (HTTP Error 404: PUGVIEW.NotFound)
10877420 (HTTP Error 404: PUGVIEW.NotFound)
355227 (HTTP Error 404: PUGVIEW.NotFound)
135433495 (HTTP Error 404: PUGVIEW.NotFound)
14980217 (HTTP Error 404: PUGVIEW.NotFound)
101931404 (HTTP Error 404: PUGVIEW.NotFound)
76964121 (HTTP Error 404: PUGVIEW.NotFound)
73416156 (HTTP Error 404: PUGVIEW.NotFound)
3084051 (HTTP Error 404: PUGVIEW.NotFound)
101702680 (HTTP Error 404: PUGVIEW.NotFound)
5

153909812 (HTTP Error 404: PUGVIEW.NotFound)
76968028 (HTTP Error 404: PUGVIEW.NotFound)
71533696 (HTTP Error 404: PUGVIEW.NotFound)
13892722 (HTTP Error 404: PUGVIEW.NotFound)
13944255 (HTTP Error 404: PUGVIEW.NotFound)
131751173 (HTTP Error 404: PUGVIEW.NotFound)
441964 (HTTP Error 404: PUGVIEW.NotFound)
124485244 (HTTP Error 404: PUGVIEW.NotFound)
3037346 (HTTP Error 404: PUGVIEW.NotFound)
719629 (HTTP Error 404: PUGVIEW.NotFound)
193993 (HTTP Error 404: PUGVIEW.NotFound)
81640219 (HTTP Error 404: PUGVIEW.NotFound)
66894 (HTTP Error 404: PUGVIEW.NotFound)
170612 (HTTP Error 404: PUGVIEW.NotFound)
787425 (HTTP Error 404: PUGVIEW.NotFound)
56682996 (HTTP Error 404: PUGVIEW.NotFound)
76962488 (HTTP Error 404: PUGVIEW.NotFound)
44576259 (HTTP Error 404: PUGVIEW.NotFound)
449137 (HTTP Error 404: PUGVIEW.NotFound)
11450478 (HTTP Error 404: PUGVIEW.NotFound)
15171724 (HTTP Error 404: PUGVIEW.NotFound)
76956080 (HTTP Error 404: PUGVIEW.NotFound)
94334244 (HTTP Error 404: PUGVIEW.NotFound)
1

NameError: name 'df_combined' is not defined

In [15]:
df.isna().sum()

Unnamed: 0                            0
zinc_id                              48
smiles                                0
cid                               64116
Canonicalized Compound            64116
Hydrogen Bond Acceptor Count      64116
Hydrogen Bond Donor Count         64116
Rotatable Bond Count              64116
Allowed IUPAC Name                64177
CAS-like Style IUPAC Name         64177
Markup IUPAC Name                 64177
Preferred IUPAC Name              64177
Systematic IUPAC Name             64177
Traditional IUPAC Name            64177
Standard InChI                    64116
Standard InChIKey                 64116
XLogP3-AA Log P                   73123
Exact Mass                        64116
Canonical SMILES                  64116
Isomeric SMILES                   64116
Polar Surface Area Topological    64116
MonoIsotopic Weight               64116
XLogP3 Log P                      95867
logp_experimental                 64116
dtype: int64

In [11]:
df['xlogp'] = df['XLogP3-AA Log P']
df['xlogp'].fillna(df['XLogP3 Log P'])
df.head(10)

,Unnamed: 0,zinc_id,smiles,cid,Canonicalized Compound,Hydrogen Bond Acceptor Count,Hydrogen Bond Donor Count,Rotatable Bond Count,Allowed IUPAC Name,CAS-like Style IUPAC Name,...,Standard InChI,Standard InChIKey,XLogP3-AA Log P,Exact Mass,Canonical SMILES,Isomeric SMILES,Polar Surface Area Topological,MonoIsotopic Weight,XLogP3 Log P,xlogp
0,0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,9850878.0,1.0,7.0,3.0,10.0,"[(1S,2R,3S,4S,6R,7R,8R,14R)-3-hydroxy-2,4,7,14...",2-[[1-[[(2R)-2-amino-3-methyl-1-oxobutyl]amino...,...,InChI=1S/C31H52N2O5S/c1-10-29(8)15-22(38-23(35...,LLYYNOVSVPBRGV-MVNKZKPCSA-N,5.3,564.359694,CC1CCC23CCC(=O)C2C1(C(CC(C(C3C)O)(C)C=C)OC(=O)...,C[C@@H]1CC[C@@]23CCC(=O)[C@H]2[C@@]1([C@@H](C[...,144.0,564.359694,NaN,5.3
1,1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,9971439.0,1.0,3.0,1.0,8.0,(2S)-2-[bis(2-chloroethyl)amino]-3-phenyl-prop...,(2S)-2-[bis(2-chloroethyl)amino]-3-phenylpropa...,...,InChI=1S/C13H17Cl2NO2/c14-6-8-16(9-7-15)12(13(...,QEGIMPLUXVXFNQ-LBPRGKRZSA-N,0.8,289.063634,C1=CC=C(C=C1)CC(C(=O)O)N(CCCl)CCCl,C1=CC=C(C=C1)C[C@@H](C(=O)O)N(CCCl)CCCl,40.5,289.063634,NaN,0.8
4,4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,ZINC000230052473,CC[C@@H]1[C@H](Cc2[nH]c(Cc3[nH]c(C[C@@H]4NC(=O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,ZINC000085875971,O=C(O)/C=C/c1cc2ccoc2cc1O[C@H]1O[C@@H](CO)[C@@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,ZINC000015206001,COc1ccc2c([nH]c3cc(O)c(C)cc32)c1C/C=C(\C)CCC=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,ZINC000150357141,CC/C=C\C/C=C\C/C=C\C/C=C\CCCCCCC(=O)O[C@@H](CO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,ZINC000150366233,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)OC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_pc = df[df.cid.notna()]

In [30]:
df_pc['xlogp'] = df_pc['xlogp'].fillna(df_pc['XLogP3 Log P'])
df_pc['xlogp'].isna().sum()

/var/folders/_d/7wdbwyr90tnbj65zx3hysd6h0000gp/T/ipykernel_11373/2507134205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pc['xlogp'] = df_pc['xlogp'].fillna(df_pc['XLogP3 Log P'])


82

In [32]:
len(df_pc[df_pc.xlogp.isna()])

82

In [33]:
len(df_pc)

20316

In [35]:
import pickle
model = pickle.load(open("clf.p", "rb"))

In [107]:
re.match('\d+\.?\d{0,4}', '2.03 LogP').group()

'2.03'

In [89]:
statistics.mean(map(lambda v: float(v), list(filter(lambda v: v, map(extract_value, data['Record']['Section'][0]['Section'][0]['Section'][0]['Information'])))))

2.13

In [114]:
get_experimental_logp(241)

2.13

In [17]:
df.to_csv('named.csv')

In [20]:
df1 = pd.read_csv('named.csv')

In [21]:
df1.notna().sum()

zinc_id                           104667
smiles                            104715
Canonicalized Compound             40599
Hydrogen Bond Acceptor Count       40599
Hydrogen Bond Donor Count          40599
Rotatable Bond Count               40599
Allowed IUPAC Name                 40538
CAS-like Style IUPAC Name          40538
Markup IUPAC Name                  40538
Preferred IUPAC Name               40538
Systematic IUPAC Name              40538
Traditional IUPAC Name             40538
Standard InChI                     40599
Standard InChIKey                  40599
Unnamed: 14                            0
Exact Mass                         40599
Canonical SMILES                   40599
Isomeric SMILES                    40599
Polar Surface Area Topological     40599
MonoIsotopic Weight                40599
cid                                40599
XLogP3-AA Log P                    31592
XLogP3 Log P                        8848
logp_experimental                   4795
dtype: int64

In [ ]:
df['xlogp'] = df['XLogP3-AA Log P']
df['xlogp'].fillna(df['XLogP3 Log P'])

In [23]:
df2 = df1[df1.cid.notna()]
df2.notna().sum()

zinc_id                           40555
smiles                            40599
Canonicalized Compound            40599
Hydrogen Bond Acceptor Count      40599
Hydrogen Bond Donor Count         40599
Rotatable Bond Count              40599
Allowed IUPAC Name                40538
CAS-like Style IUPAC Name         40538
Markup IUPAC Name                 40538
Preferred IUPAC Name              40538
Systematic IUPAC Name             40538
Traditional IUPAC Name            40538
Standard InChI                    40599
Standard InChIKey                 40599
Unnamed: 14                           0
Exact Mass                        40599
Canonical SMILES                  40599
Isomeric SMILES                   40599
Polar Surface Area Topological    40599
MonoIsotopic Weight               40599
cid                               40599
XLogP3-AA Log P                   31592
XLogP3 Log P                       8848
logp_experimental                  4795
dtype: int64

## Create a combined logp column
There are not enough experimental data points to build a model, so while they are used preferentially, the data will be backfilled using predicted data, first from XLogP3 then from XLogP3 AA (which is an atomic contribution method similar to Wildman and Crippen)

In [24]:
df2['logp'] = df2['logp_experimental']
df2.notna().sum()

/var/folders/_d/7wdbwyr90tnbj65zx3hysd6h0000gp/T/ipykernel_9854/3883517327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['logp'] = df2['logp_experimental']


zinc_id                           40555
smiles                            40599
Canonicalized Compound            40599
Hydrogen Bond Acceptor Count      40599
Hydrogen Bond Donor Count         40599
Rotatable Bond Count              40599
Allowed IUPAC Name                40538
CAS-like Style IUPAC Name         40538
Markup IUPAC Name                 40538
Preferred IUPAC Name              40538
Systematic IUPAC Name             40538
Traditional IUPAC Name            40538
Standard InChI                    40599
Standard InChIKey                 40599
Unnamed: 14                           0
Exact Mass                        40599
Canonical SMILES                  40599
Isomeric SMILES                   40599
Polar Surface Area Topological    40599
MonoIsotopic Weight               40599
cid                               40599
XLogP3-AA Log P                   31592
XLogP3 Log P                       8848
logp_experimental                  4795
logp                               4795


In [26]:
df2['logp'].fillna(df2['XLogP3 Log P'], inplace=True)
df2.notna().sum()

/usr/local/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


zinc_id                           40555
smiles                            40599
Canonicalized Compound            40599
Hydrogen Bond Acceptor Count      40599
Hydrogen Bond Donor Count         40599
Rotatable Bond Count              40599
Allowed IUPAC Name                40538
CAS-like Style IUPAC Name         40538
Markup IUPAC Name                 40538
Preferred IUPAC Name              40538
Systematic IUPAC Name             40538
Traditional IUPAC Name            40538
Standard InChI                    40599
Standard InChIKey                 40599
Unnamed: 14                           0
Exact Mass                        40599
Canonical SMILES                  40599
Isomeric SMILES                   40599
Polar Surface Area Topological    40599
MonoIsotopic Weight               40599
cid                               40599
XLogP3-AA Log P                   31592
XLogP3 Log P                       8848
logp_experimental                  4795
logp                              11006


In [27]:
df2['logp'].fillna(df2['XLogP3-AA Log P'], inplace=True)
df2.notna().sum()

zinc_id                           40555
smiles                            40599
Canonicalized Compound            40599
Hydrogen Bond Acceptor Count      40599
Hydrogen Bond Donor Count         40599
Rotatable Bond Count              40599
Allowed IUPAC Name                40538
CAS-like Style IUPAC Name         40538
Markup IUPAC Name                 40538
Preferred IUPAC Name              40538
Systematic IUPAC Name             40538
Traditional IUPAC Name            40538
Standard InChI                    40599
Standard InChIKey                 40599
Unnamed: 14                           0
Exact Mass                        40599
Canonical SMILES                  40599
Isomeric SMILES                   40599
Polar Surface Area Topological    40599
MonoIsotopic Weight               40599
cid                               40599
XLogP3-AA Log P                   31592
XLogP3 Log P                       8848
logp_experimental                  4795
logp                              40449


In [28]:
df2.to_csv('chem_data.csv')

In [32]:
df3 = df2.drop('Unnamed: 14', axis=1)

In [33]:
df3.columns

Index(['zinc_id', 'smiles', 'Canonicalized Compound',
       'Hydrogen Bond Acceptor Count', 'Hydrogen Bond Donor Count',
       'Rotatable Bond Count', 'Allowed IUPAC Name',
       'CAS-like Style IUPAC Name', 'Markup IUPAC Name',
       'Preferred IUPAC Name', 'Systematic IUPAC Name',
       'Traditional IUPAC Name', 'Standard InChI', 'Standard InChIKey',
       'Exact Mass', 'Canonical SMILES', 'Isomeric SMILES',
       'Polar Surface Area Topological', 'MonoIsotopic Weight', 'cid',
       'XLogP3-AA Log P', 'XLogP3 Log P', 'logp_experimental', 'logp'],
      dtype='object')

In [34]:
df3.to_csv('chem_data.csv')